In [2]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
#ws = Workspace.get(name='quick-starts-ws-137411')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-137411
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-137411


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = 'my-compute'
try:
    cpu_cluster = ComputeTarget(workspace=ws, name= compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, compute_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)



Found existing cluster, use it.

Jobrunning.........................................
Running


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler

### YOUR CODE HERE ###
ps = RandomParameterSampling({
    "--C": choice (1.0,2.0,3.0,4.0),
    "--max_iter":choice (50,75,100,125,150)
})


# Specify a Policy
policy = BanditPolicy (evaluation_interval=1, slack_factor=0.2)

if 'training' not in os.listdir():
    os.mkdir('./training')
    
import shutil
shutil.copy('train.py','./training')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './training',
            compute_target= cpu_cluster,
             entry_script = 'train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,hyperparameter_sampling=ps,
                                     policy= policy,
                                     primary_metric_name = "Accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 20,
                                     max_concurrent_runs=3)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hd_run = exp.submit(hyperdrive_config)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [42]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print ('Best Run Id: ' , best_run.id)
print('Accuracy: ', best_run_metrics ['Accuracy'])
print('Regularization Strength: ', best_run_metrics ['Regularization Strength:'])
print('Max Iterations: ', best_run_metrics ['Max iterations:'])


model = best_run.register_model (
                                 model_name= 'project1model',
                                 model_path = './',
                                 properties = {'Accuracy': best_run_metrics ['Accuracy'],
                                            'Regularization Strength': best_run_metrics ['Regularization Strength:'],
                                            'Max Iterations': best_run_metrics ['Max iterations:']})

Best Run Id:  HD_17be9b5a-9d96-4e9a-af4b-8b6efd200efc_1
Accuracy:  0.9128195696155968
Regularization Strength:  4.0
Max Iterations:  100


In [44]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
dset = TabularDatasetFactory.from_delimited_files(data)

In [45]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dset)

In [58]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=dset,
    compute_target = cpu_cluster,
    label_column_name='y',
    n_cross_validations=5)

In [59]:
# Submit your automl run

### YOUR CODE HERE ###
ws = Workspace.from_config()
AML_experiment = Experiment(ws, "AML_exp")
AML_run = AML_experiment.submit(automl_config,show_output = True)
RunDetails(AML_run).show()

ConfigException: ConfigException:
	Message: Invalid argument(s) 'primary_metric' specified. Supported value(s): 'norm_macro_recall, accuracy, average_precision_score_weighted, precision_score_weighted, AUC_weighted'.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Invalid argument(s) 'primary_metric' specified. Supported value(s): 'norm_macro_recall, accuracy, average_precision_score_weighted, precision_score_weighted, AUC_weighted'.",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "primary_metric",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run,fitted_model= AML_run.get_output()
print(best_run)
print(fitted_model)

best_run_metrics = best_run.get_metrics()
for name in best_run_metrics:
    metric = best_run_metrics[name]
    print(name,metric)